# LLM Route Feature Editor (Clean)

Minimal notebook focused on preparing interpretable route features, prompting an LLM, and applying its transformation plan.

In [1]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub


In [2]:
load_dotenv()
hf_token = os.getenv("HFT")
assert hf_token, "HFT token missing in .env"

df = pd.read_csv("/Users/eugeneleach/code/Eugle3/cycle_more/Notebooks/UK_Engineered_Data.csv")
print(f"Loaded {len(df)} routes")


Loaded 7717 routes


In [3]:
df.iloc[6]

Unnamed: 0.1                                                  6
Unnamed: 0                                                   56
id                                                      1959833
name                                  East Kilkenny Cycle Route
distance_m                                                  6.7
duration_s                                                  1.3
ascent_m                                                    0.0
descent_m                                                   0.0
steps                                                         2
turns                                                         0
Asphalt                                                   100.0
Unknown                                                     0.0
Paved                                                       0.0
Compacted Gravel                                            0.0
Wood                                                        0.0
Gravel                                  

In [4]:
def extract_interpretable_features(row):
    """
    Converts a full engineered feature row into an interpretable, LLM-friendly
    feature dictionary (Option B: grouped/aggregated features).
    """

    features = {
        "distance_m": float(row["distance_m"]),
        "duration_s": float(row["duration_s"]),
        "ascent_m": float(row["ascent_m"]),
        "descent_m": float(row["descent_m"])
    }

    surface_map = {
        "paved_percent": ["Asphalt", "Paved", "Concrete"],
        "gravel_percent": ["Gravel", "Compacted Gravel"],
        "dirt_percent": ["Dirt", "Unpaved", "Ground"],
        "grass_percent": ["Grass", "Grass Paver"],
        "other_percent": ["Wood", "Metal", "Sand", "Paving Stones", "Unknown"],
    }
    # The total percentage of the route that is paved (asphalt, paved, concrete) for example
    surface_profile = {k: float(sum(row[col] for col in cols)) for k, cols in surface_map.items()}

    hill_map = {
        "flat_percent": ["flat (0%)"],
        "gentle_uphill_percent": ["uphill_gentle (0% to 3%)"],
        "moderate_uphill_percent": ["uphill_moderate (3% to 5%)"],
        "steep_uphill_percent": ["uphill_steep (5% to 7%)", "uphill_very_steep (7% to 10%)", "uphill_extreme (>10%)"],
        "gentle_downhill_percent": ["downhill_gentle (-5% to 0%)"],
        "steep_downhill_percent": ["downhill_moderate (-7% to -5%)", "downhill_steep (-10% to -7%)", "downhill_very_steep (-15% to -10%)", "downhill_extreme (<-15%)"],
    }
    hill_profile = {k: float(sum(row[col] for col in cols)) for k, cols in hill_map.items()}

    route_shape = {
        "turns": float(row["turns"]),
        "steps": float(row["steps"]),
        "turn_density": float(row["Turn_Density"]),
        "avg_speed": float(row["Average_Speed"]),
    }

    return {
        "distance_m": features["distance_m"],
        "duration_s": features["duration_s"],
        "ascent_m": features["ascent_m"],
        "descent_m": features["descent_m"],
        "surface_profile": surface_profile,
        "hill_profile": hill_profile,
        "route_shape": route_shape,
    }


In [5]:
route_edit_prompt = PromptTemplate(
    input_variables=["features", "instruction"],
    template="""
You are an assistant that modifies cycling route features based on a user's instruction.

You will receive:
1. An interpretable cycling route feature dictionary.
2. A natural-language instruction from the user.

Your job:
- Understand the user's intent.
- Create a JSON object describing how each feature should be changed.
- Only modify features that are relevant to the user's request.
- Do NOT produce any explanation or commentary — output JSON ONLY.

The JSON format must be:

{{
  "distance_m": {{"operation": "<set/add/multiply/none>", "value": <number>}},
  "duration_s": {{"operation": "<set/add/multiply/none>", "value": <number>}},
  "ascent_m": {{"operation": "<set/add/multiply/none>", "value": <number>}},
  "descent_m": {{"operation": "<set/add/multiply/none>", "value": <number>}},
  "surface_profile": {{
    "paved_percent": {{"operation": "<set/add/multiply/none>", "value": <number>}},
    "gravel_percent": {{"operation": "<set/add/multiply/none>", "value": <number>}},
    "dirt_percent": {{"operation": "<set/add/multiply/none>", "value": <number>}},
    "grass_percent": {{"operation": "<set/add/multiply/none>", "value": <number>}},
    "other_percent": {{"operation": "<set/add/multiply/none>", "value": <number>}}
  }},
  "hill_profile": {{
    "flat_percent": {{"operation": "<set/add/multiply/none>", "value": <number>}},
    "gentle_uphill_percent": {{"operation": "<set/add/multiply/none>", "value": <number>}},
    "moderate_uphill_percent": {{"operation": "<set/add/multiply/none>", "value": <number>}},
    "steep_uphill_percent": {{"operation": "<set/add/multiply/none>", "value": <number>}},
    "gentle_downhill_percent": {{"operation": "<set/add/multiply/none>", "value": <number>}},
    "steep_downhill_percent": {{"operation": "<set/add/multiply/none>", "value": <number>}}
  }},
  "route_shape": {{
    "turns": {{"operation": "<set/add/multiply/none>", "value": <number>}},
    "steps": {{"operation": "<set/add/multiply/none>", "value": <number>}},
    "turn_density": {{"operation": "<set/add/multiply/none>", "value": <number>}},
    "avg_speed": {{"operation": "<set/add/multiply/none>", "value": <number>}}
  }}
}}

Rules:
- Use "set" to directly assign a value.
- Use "add" or "multiply" for relative changes.
- Use "none" to leave a field unchanged.
- Keep values within realistic bounds (e.g., paved_percent between 0 and 100).
- Ensure surface_profile and hill_profile percentages sum to ~100 each.
- Keep turns and steps non-negative.
"""
)


In [6]:
#op dict {"operation": "<set/add/multiply/none>", "value": <number>}
def apply_operation(original_value, op_dict):
    operation = op_dict.get("operation", "none")
    value = op_dict.get("value", 0)
    if operation == "none":
        return original_value
    if operation == "set":
        return float(value)
    if operation == "add":
        return float(original_value) + float(value)
    if operation == "multiply":
        return float(original_value) * float(value)
    return original_value


def apply_transformation_plan(original_features, plan):
    updated = {}
    for key in ["distance_m", "duration_s", "ascent_m", "descent_m"]:
        updated[key] = apply_operation(original_features[key], plan[key])

    updated_surface = {
        k: apply_operation(original_features["surface_profile"][k], plan["surface_profile"][k])
        for k in original_features["surface_profile"].keys()
    }
    total = sum(updated_surface.values())
    if total > 0:
        updated_surface = {k: (v / total) * 100 for k, v in updated_surface.items()}

    updated_hill = {
        k: apply_operation(original_features["hill_profile"][k], plan["hill_profile"][k])
        for k in original_features["hill_profile"].keys()
    }
    total_h = sum(updated_hill.values())
    if total_h > 0:
        updated_hill = {k: (v / total_h) * 100 for k, v in updated_hill.items()}

    updated_shape = {
        k: apply_operation(original_features["route_shape"][k], plan["route_shape"][k])
        for k in original_features["route_shape"].keys()
    }
    updated_shape["turns"] = max(0, updated_shape["turns"])
    updated_shape["steps"] = max(0, updated_shape["steps"])

    updated["surface_profile"] = updated_surface
    updated["hill_profile"] = updated_hill
    updated["route_shape"] = updated_shape
    return updated


In [7]:
import os

In [8]:
GKEY = os.getenv("GEMINIKEY")

In [ ]:
# Build LLM client for conversational task
llm = HuggingFaceHub(
    repo_id="TeichAI/Qwen3-4B-Thinking-2507-Gemini-2.5-Flash-Distill-GGUF",
    task="conversational",
    huggingfacehub_api_token=hf_token,
    model_kwargs={
        "temperature": 0.1,
        "max_new_tokens": 512,
    },
)

# # Patch InferenceClient to match old .post interface expected by HuggingFaceHub
# import json as _json

# def _post_patch(json=None, task=None):
#     prompt = json["inputs"]
#     params = json.get("parameters", {})
#     res = llm.client.text_generation(prompt, **params)
#     text = res if isinstance(res, str) else res.generated_text
#     return _json.dumps([{ "generated_text": text }]).encode()

# llm.client.post = _post_patch
# print("✅ LLM ready (patched InferenceClient.post)")


✅ LLM ready (patched InferenceClient.post)


In [14]:
# Sample a route, build prompt, and apply plan
row = df.sample(1).iloc[0]
features = extract_interpretable_features(row)

instruction = "Make it about 2x longer, mostly paved, fewer turns"

prompt = route_edit_prompt.format(
    features=json.dumps(features, indent=2),
    instruction=instruction,
)
prompt

'\nYou are an assistant that modifies cycling route features based on a user\'s instruction.\n\nYou will receive:\n1. An interpretable cycling route feature dictionary.\n2. A natural-language instruction from the user.\n\nYour job:\n- Understand the user\'s intent.\n- Create a JSON object describing how each feature should be changed.\n- Only modify features that are relevant to the user\'s request.\n- Do NOT produce any explanation or commentary — output JSON ONLY.\n\nThe JSON format must be:\n\n{\n  "distance_m": {"operation": "<set/add/multiply/none>", "value": <number>},\n  "duration_s": {"operation": "<set/add/multiply/none>", "value": <number>},\n  "ascent_m": {"operation": "<set/add/multiply/none>", "value": <number>},\n  "descent_m": {"operation": "<set/add/multiply/none>", "value": <number>},\n  "surface_profile": {\n    "paved_percent": {"operation": "<set/add/multiply/none>", "value": <number>},\n    "gravel_percent": {"operation": "<set/add/multiply/none>", "value": <number

In [15]:
raw_plan = llm.invoke(prompt)

StopIteration: 

In [ ]:

raw_plan = llm.invoke(prompt)
plan = json.loads(raw_plan)
updated = apply_transformation_plan(features, plan)
print("Original features:", json.dumps(features, indent=2))
print("Plan:", json.dumps(plan, indent=2))
print("Updated features:", json.dumps(updated, indent=2))
